# On-The-Fly (OTF) Data Reduction

This notebook shows how to use `dysh` to calibrate and grid an *On-The-Fly* (OTF) observation. See Mangum et al. (2007) in https://ui.adsabs.harvard.edu/abs/2007A%26A...474..679M for the background on this method.

This more complex observation is not reduced with one simple command in dysh, but is usually two steps: 

1. The first step is calibration, where the calibrated spectra are written to an SDFITS file. This can be done in dysh, usually in a loop, one spectrum a time.

2. The second step takes these spectra and write
them to a FITS cube. For this there are several third-party tools available, there is nothing in dysh yet to cover this.

In [ ]:
from pathlib import Path
import astropy.units as u
from dysh.util.download import from_url
from dysh.fits.gbtfitsload import GBTFITSLoad
from dysh.util.files import dysh_data

from dysh.spectra import ScanBlock

## Data Retrieval

Download the example SDFITS data, if necessary. or rely on $DYSH_DATA

In [ ]:
filename = dysh_data(example='mapping-L/data/TGBT17A_506_11.raw.vegas')    
print(filename)

## Data Loading

Next, we use `GBTFITSLoad` to load the data, and then its `summary` method to inspect its contents. This takes a few seconds.

In [ ]:
sdfits = GBTFITSLoad(filename)

In [ ]:
sdfits.summary()

In this particular observation the OTF slew over the galaxy NGC6946 in scans 14-26, followed by an Off position. Each **SCAN**, in this case, corresponds to a row, with 61 integrations as the telescope slews slowly over the sky.
Instead of the classical OnOff reduction with `getps`, we can use `getsigref` to refer to scan 27 as the common Off.


## Data Reduction



In [ ]:
ifnum=0        # needs to be 0, where the 21cm signal is
plnum=[0,1]    # pick 0 and 1, they will be averaged. or just pick one
fdnum=0        # only one feed in this dataset
nint = 61      # nunber of integrations per scan

scans = list(range(14,27))
ref = 27

In [ ]:
%%time 

sb = ScanBlock()

for s in scans:
    for p in plnum:
        print(f"Working on scan {s} pol {p}")
        sb1 =  sdfits.getsigref(scan=s, ref=ref, fdnum=fdnum, ifnum=ifnum, plnum=p)[0]
        #  @todo baseline subtraction
        sb.append(sb1)
    
sb.write("otf1.fits", overwrite=True)    #  300 ms

This calibration step takes about 2.5 minutes, since there are  1586 (13\*61\*2) spectra written.

This is without baseline subtraction. To be rewritten.

## Gridding

The most commonly used task for this is `gbtgridder`, which is not part of dysh.  

Here's a super short blurb how to get it, with
the note that it is important to get the correct release branch.   This was the situation spring 2025, and it may change, so be sure to be in touch with the developers for recommendations.

```
      git clone -b release_3.0  https://github.com/GreenBankObservatory/gbtgridder
      pip install -e gbtgridder
```

After installation, either from the shell, or from the notebook, one can grid

```
      gbtgridder --size 32 32  --channels 500:3500 -o otf1 --clobber --auto otf1.fits
```

this process will create `test2_cube.fits` andd `test2_weight.fits`

We should show the FITS header, and we should plot a spectrum.



In [ ]:
!gbtgridder --size 32 32  --channels 500:3500 -o otf1 --clobber --auto otf1.fits

In [ ]:
ls -l

## Viewing the gridded cube

The cube should be 32 x 32 x 3001 x 1 

Should check if the gridder can handle NAXIS4=2, but dysh may not handle this. Future feature?

In [ ]:
from astropy.io import fits


In [ ]:
cube = 'otf1_cube.fits'

hdu = fits.open(cube)
header = hdu[0].header
print(header)



In [ ]:
import matplotlib.pyplot as plt

data = hdu[0].data
slice = data[0,1500]
#  no WCS, but north is up this way and east to the left
plt.imshow(slice, origin='lower')

In [ ]:
spec = data[0,:,16,16]

plt.plot(spec)
plt.xlabel("Channel")
plt.ylabel(header["BUNIT"])
plt.title(header["OBJECT"]);

## SpectralCube

another teachable moment?


In [ ]:
try:
    from spectral_cube import SpectralCube

    my_cube = SpectralCube.read(cube).with_spectral_unit(u.km/u.s)

except:
    print("alas, there is no SpectralCube in your python. You could try:   pip install spectral_cube")

## Viewing interactively

Here we would be leaveing the notebook and using your shell environment. Use at your own risk, the commands have been commented out as not to hang the automated notebook checkers.

A note of caution if you use CARTA: the cell needs to remain running while viewing. Kill it with the "interrupt the kernel".

In [ ]:
# or go the command line way:

#!ds9 otf1_cube.fits

In [ ]:
# careful, carta leaves the cell running as long as you want to view the image After that:  interrupt the kernel.

#!carta otf1_cube.fits